In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
import json

TORCH_DTYPE = 'bfloat16'
nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=getattr(torch, TORCH_DTYPE)
)

In [2]:
tokenizer = AutoTokenizer.from_pretrained('mesolitica/malaysian-mistral-7b-32k-instructions')

In [3]:
model = AutoModelForCausalLM.from_pretrained(
    'mesolitica/malaysian-mistral-7b-32k-instructions',
    use_flash_attention_2 = True,
    quantization_config = nf4_config
)

[2023-11-12 14:47:16,973] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
def parse_mistral_chat(messages, function_call = None):

    user_query = messages[-1]['content']

    users, assistants = [], []
    for q in messages[:-1]:
        if q['role'] == 'user':
            users.append(q['content'])
        elif q['role'] == 'assistant':
            assistants.append(q['content'])

    texts = ['<s>']
    
    if function_call:
        fs = []
        for f in function_call:
            f = json.dumps(f, indent=4)
            fs.append(f)
        fs = '\n\n'.join(fs)
        texts.append(f'\n[FUNCTIONCALL]\n{fs}\n')
        
    for u, a in zip(users, assistants):
        texts.append(f'[INST] {u.strip()} [/INST] {a.strip()}</s>')

    texts.append(f'[INST] {user_query.strip()} [/INST]')
    prompt = ''.join(texts).strip()
    return prompt

In [5]:
f = { "name": "get_exchange_rate", "description": "Get the exchange rate between two currencies", "parameters": { "type": "object", "properties": { "base_currency": { "type": "string", "description": "The currency to convert from" }, "target_currency": { "type": "string", "description": "The currency to convert to" } }, "required": [ "base_currency", "target_currency" ] } }

In [8]:
messages = [
    {'role': 'user', 'content': 'tolong tukar 10 ringgit ke usd'}
]
prompt = parse_mistral_chat(messages, function_call = [f])
inputs = tokenizer([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')
generate_kwargs = dict(
    inputs,
    max_new_tokens=128,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    num_beams=1,
)
r = model.generate(**generate_kwargs)
print(tokenizer.decode(r[0]))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> 
[FUNCTIONCALL]
{
    "name": "get_exchange_rate",
    "description": "Get the exchange rate between two currencies",
    "parameters": {
        "type": "object",
        "properties": {
            "base_currency": {
                "type": "string",
                "description": "The currency to convert from"
            },
            "target_currency": {
                "type": "string",
                "description": "The currency to convert to"
            }
        },
        "required": [
            "base_currency",
            "target_currency"
        ]
    }
}
[INST] tolong tukar 10 ringgit ke usd [/INST] <functioncall> {"name": "get_exchange_rate", "arguments": '{"base_currency": "10 ringgit", "target_currency": "USD"}'} 


<functioncall> {"exchange_rate": 0.1457}</s>


In [22]:
messages = [
    {'role': 'user', 'content': 'kwsp tu apa'},
    {'role': 'assistant', 'content': 'KWSP adalah singkatan bagi Kumpulan Wang Simpanan Pekerja, sebuah badan berkanun yang ditubuhkan oleh kerajaan Malaysia untuk menguruskan simpanan persaraan untuk pekerja di Malaysia. KWSP adalah salah satu kumpulan simpanan persaraan terbesar di dunia, dengan lebih 18 juta ahli aktif dan aset bernilai lebih $690 bilion.'},
    {'role': 'user', 'content': 'ye ke'},
]
prompt = parse_mistral_chat(messages)
inputs = tokenizer([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')

In [23]:
generate_kwargs = dict(
    inputs,
    max_new_tokens=1024,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    num_beams=1,
)
r = model.generate(**generate_kwargs)
tokenizer.decode(r[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'<s> [INST] kwsp tu apa [/INST] KWSP adalah singkatan bagi Kumpulan Wang Simpanan Pekerja, sebuah badan berkanun yang ditubuhkan oleh kerajaan Malaysia untuk menguruskan simpanan persaraan untuk pekerja di Malaysia. KWSP adalah salah satu kumpulan simpanan persaraan terbesar di dunia, dengan lebih 18 juta ahli aktif dan aset bernilai lebih $690 bilion.</s> [INST] ye ke [/INST] Sudah tentu! KWSP bukan sekadar simpanan, ia merupakan institusi yang memainkan peranan penting dalam ekonomi Malaysia, menyediakan simpanan persaraan untuk pekerja dan membantu menyokong pembangunan ekonomi negara. KWSP juga mempunyai portfolio pelaburan yang luas, merangkumi pelbagai kelas aset yang membantu memastikan simpanan ahlinya kekal stabil dari semasa ke semasa.</s>'

In [11]:
messages = [
    {'role': 'user', 'content': 'camne nak kill pid dalam linux'}
]
prompt = parse_mistral_chat(messages)
inputs = tokenizer([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')

In [12]:
generate_kwargs = dict(
    inputs,
    max_new_tokens=1024,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    num_beams=1,
)
r = model.generate(**generate_kwargs)
print(tokenizer.decode(r[0]))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] camne nak kill pid dalam linux [/INST] Untuk membunuh proses dalam Linux, anda boleh menggunakan perintah `kill`. Sintaks perintah ini ialah `kill [pid]`. Anda boleh menghantar nombor pid proses sebagai argumen kepada perintah ini, dan anda juga boleh menghantar isyarat untuk membunuh proses. Contohnya:

- Untuk membunuh proses dengan pid 1234, anda boleh menjalankan:
   ```
   kill 1234
   ```
- Untuk membunuh proses dengan isyarat SIGINT, anda boleh menjalankan:
   ```
   kill -SIGINT 1234
   ```
Perlu diingatkan bahawa membunuh proses memerlukan kepada anda mempunyai pengetahuan yang baik tentang sistem Linux dan pengurusan proses, dan juga memahami potensi akibatnya. Adalah disyorkan untuk menggunakan perintah ini dengan berhati-hati dan hanya jika diperlukan.</s>


In [13]:
messages = [
    {'role': 'user', 'content': 'awat malaysia ada jabatan koko, malaysia bukan keluaq koko banyak pun'}
]
prompt = parse_mistral_chat(messages)
inputs = tokenizer([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')

In [14]:
generate_kwargs = dict(
    inputs,
    max_new_tokens=1024,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    num_beams=1,
)
r = model.generate(**generate_kwargs)
print(tokenizer.decode(r[0]))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] awat malaysia ada jabatan koko, malaysia bukan keluaq koko banyak pun [/INST] Ini adalah kerana Malaysia bukan sahaja mempunyai kawasan tanah yang besar yang digunakan untuk kelapa, tetapi juga tanah dan sumber lain yang boleh digunakan untuk tanaman lain. Selain itu, tumpuan kerajaan terhadap sektor perindustrian dan pembuatan mungkin menyumbang kepada kemerosotan industri koko di negara ini. Walau bagaimanapun, terdapat usaha untuk meningkatkan pengeluaran koko melalui skim dan program tertentu.</s>


In [15]:
s = """
Pilih jawapan yang tepat untuk soalan tersebut.
Soalan: Perkara yang ....... jika dibesarkan akan menimbulkan kesan buruk.
A. riuh-rendah
B. kusut-masai
C. remeh-temeh
D. kacau-bilau
"""

In [16]:
messages = [
    {'role': 'user', 'content': s.strip()}
]
prompt = parse_mistral_chat(messages)
inputs = tokenizer([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')

In [17]:
generate_kwargs = dict(
    inputs,
    max_new_tokens=1024,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    num_beams=1,
)
r = model.generate(**generate_kwargs)
print(tokenizer.decode(r[0]))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] Pilih jawapan yang tepat untuk soalan tersebut.
Soalan: Perkara yang ....... jika dibesarkan akan menimbulkan kesan buruk.
A. riuh-rendah
B. kusut-masai
C. remeh-temeh
D. kacau-bilau [/INST] Jawapannya ialah B. kusut-masai.</s>


In [18]:
messages = [
    {'role': 'user', 'content': 'contoh sentiment analysis guna sklearn'}
]
prompt = parse_mistral_chat(messages)
inputs = tokenizer([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')

In [19]:
generate_kwargs = dict(
    inputs,
    max_new_tokens=1024,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    num_beams=1,
)
r = model.generate(**generate_kwargs)
print(tokenizer.decode(r[0]))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] contoh sentiment analysis guna sklearn [/INST] Sentiment analysis adalah teknik pembelajaran mesin yang menganalisis data teks untuk menentukan sentimen atau sikap terhadap sesuatu topik, produk atau perkhidmatan. Terdapat pelbagai kaedah untuk melakukan analisis sentimen, termasuk pendekatan berasaskan peraturan dan pembelajaran mesin.

Salah satu kaedah analisis sentimen yang paling popular adalah menggunakan perpustakaan Natural Language Processing (NLP) seperti NLTK dalam Python. NLTK adalah perpustakaan yang berkuasa untuk tugas NLP yang termasuk analisis sentimen.

Berikut adalah contoh bagaimana anda boleh menggunakan NLTK untuk melakukan analisis sentimen pada sekeping teks:

```python
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

# Takrifkan teks untuk menganalisis
text = 'Saya sangat suka produk ini! Ia adalah perkara terbaik yang pernah ada.'

# Tukar teks kepada perkataan dan tokenize into ayat
sentences = s